<a href="https://colab.research.google.com/github/sprucesunday777/Sentimental-analysis-on-fiverr/blob/main/Fiverr12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
from transformers import T5Tokenizer, T5ForSequenceClassification, T5ForConditionalGeneration
import torch

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [3]:
df = pd.read_csv('/content/processed_Hotel_review.csv')

In [4]:
df.head()

,Unnamed: 0,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Reviews_Clean,Score
0,278594,Catalonia Atenas,Room smelt of smoke despite asking for no smo...,Spa and spa staff lovely Choice of breakfast ...,7.1,room smelt of smoke despite asking for no smo...,1
1,214232,Shaftesbury Premier London Paddington,No Negative,Lovely staff comfortable room Comfy beds and ...,9.6,no negative lovely staff comfortable room comf...,1
2,267990,Corendon Vitality Hotel Amsterdam,No Negative,Everything hotel and staff were very good and...,10.0,no negative everything hotel and staff were ve...,1
3,85270,Grange White Hall Hotel,The bar area was shabby and disorganised The ...,Reception on arrival and departure were very ...,6.3,the bar area was shabby and disorganised the ...,1
4,379076,Leonardo Hotel Vienna,I think wi fi should be offered for free,No Positive,10.0,i think wi fi should be offered for freeno po...,1


In [5]:
df[(df['Score'] == 0) & (df['Reviewer_Score'] >= 5.9)]

,Unnamed: 0,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Reviews_Clean,Score


In [6]:
display(df['Negative_Review'][0])
display(df['Positive_Review'][0])
display(df['Reviews_Clean'][0])

' Room smelt of smoke despite asking for no smoking room Room quite small staff in breakfast room rude'

' Spa and spa staff lovely Choice of breakfast good Close to metro'

' room smelt of smoke despite asking for no smoking room room quite small staff in breakfast room rude spa and spa staff lovely choice of breakfast good close to metro'

In [7]:
display(df['Negative_Review'][11118])
display(df['Positive_Review'][11118])
display(df['Reviews_Clean'][11118])

' Hidden costs and non transparency'

' Facilities'

' hidden costs and non transparency facilities'

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21620 entries, 0 to 21619
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       21620 non-null  int64  
 1   Hotel_Name       21620 non-null  object 
 2   Negative_Review  21620 non-null  object 
 3   Positive_Review  21620 non-null  object 
 4   Reviewer_Score   21620 non-null  float64
 5   Reviews_Clean    21619 non-null  object 
 6   Score            21620 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1.2+ MB


In [9]:
copy_df = df[['Negative_Review', 'Positive_Review']]

In [10]:
copy_df['Positive_Review'] = copy_df['Positive_Review'].apply(lambda x: x.replace("No Positive", ""))

<ipython-input-10-9c3954fd838f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df['Positive_Review'] = copy_df['Positive_Review'].apply(lambda x: x.replace("No Positive", ""))


In [11]:
copy_df['Negative_Review'] = copy_df['Negative_Review'].apply(lambda x: x.replace("No Negative", ""))

<ipython-input-11-d5f677f2ee78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df['Negative_Review'] = copy_df['Negative_Review'].apply(lambda x: x.replace("No Negative", ""))


In [12]:
copy_df

,Negative_Review,Positive_Review
0,Room smelt of smoke despite asking for no smo...,Spa and spa staff lovely Choice of breakfast ...
1,,Lovely staff comfortable room Comfy beds and ...
2,,Everything hotel and staff were very good and...
3,The bar area was shabby and disorganised The ...,Reception on arrival and departure were very ...
4,I think wi fi should be offered for free,
...,...,...
21615,Bed could have been better Staff could have b...,
21616,Wifi was bad Hausekeeping was not enough Loca...,The room was large wide and good
21617,Very dirty hotel noisy AC very unfriendly staff,Good lcoation
21618,Selection of breakfast items is poor,


In [13]:
final_df = copy_df['Negative_Review'] + copy_df['Positive_Review']

In [14]:
final_df

0         Room smelt of smoke despite asking for no smo...
1         Lovely staff comfortable room Comfy beds and ...
2         Everything hotel and staff were very good and...
3         The bar area was shabby and disorganised The ...
4                 I think wi fi should be offered for free
                               ...                        
21615     Bed could have been better Staff could have b...
21616     Wifi was bad Hausekeeping was not enough Loca...
21617     Very dirty hotel noisy AC very unfriendly sta...
21618                 Selection of breakfast items is poor
21619     The hotel does not merit four stars There is ...
Length: 21620, dtype: object

In [15]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
#model = T5ForSequenceClassification.from_pretrained('t5-large', return_dict=True)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
reviews = list(final_df)

In [17]:
display(len(reviews))
len(df)

21620

21620

In [18]:
from transformers  import pipeline

In [19]:
classifier = pipeline(task='sentiment-analysis')#, model=model, tokenizer=tokenizer)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [20]:
['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTBigCodeForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTNeoXForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceClassification', 'LayoutLMv2ForSequenceClassification', 'LayoutLMv3ForSequenceClassification', 'LEDForSequenceClassification', 'LiltForSequenceClassification', 'LlamaForSequenceClassification', 'LongformerForSequenceClassification', 'LukeForSequenceClassification', 'MarkupLMForSequenceClassification', 'MBartForSequenceClassification', 'MegaForSequenceClassification', 'MegatronBertForSequenceClassification', 'MistralForSequenceClassification', 'MobileBertForSequenceClassification', 'MPNetForSequenceClassification', 'MptForSequenceClassification', 'MraForSequenceClassification', 'MT5ForSequenceClassification', 'MvpForSequenceClassification', 'NezhaForSequenceClassification', 'NystromformerForSequenceClassification', 'OpenLlamaForSequenceClassification', 'OpenAIGPTForSequenceClassification', 'OPTForSequenceClassification', 'PerceiverForSequenceClassification', 'PersimmonForSequenceClassification', 'PLBartForSequenceClassification', 'QDQBertForSequenceClassification', 'ReformerForSequenceClassification', 'RemBertForSequenceClassification', 'RobertaForSequenceClassification', 'RobertaPreLayerNormForSequenceClassification', 'RoCBertForSequenceClassification', 'RoFormerForSequenceClassification', 'SqueezeBertForSequenceClassification', 'T5ForSequenceClassification', 'TapasForSequenceClassification', 'TransfoXLForSequenceClassification', 'UMT5ForSequenceClassification', 'XLMForSequenceClassification', 'XLMRobertaForSequenceClassification', 'XLMRobertaXLForSequenceClassification', 'XLNetForSequenceClassification', 'XmodForSequenceClassification', 'YosoForSequenceClassification']

['AlbertForSequenceClassification',
 'BartForSequenceClassification',
 'BertForSequenceClassification',
 'BigBirdForSequenceClassification',
 'BigBirdPegasusForSequenceClassification',
 'BioGptForSequenceClassification',
 'BloomForSequenceClassification',
 'CamembertForSequenceClassification',
 'CanineForSequenceClassification',
 'LlamaForSequenceClassification',
 'ConvBertForSequenceClassification',
 'CTRLForSequenceClassification',
 'Data2VecTextForSequenceClassification',
 'DebertaForSequenceClassification',
 'DebertaV2ForSequenceClassification',
 'DistilBertForSequenceClassification',
 'ElectraForSequenceClassification',
 'ErnieForSequenceClassification',
 'ErnieMForSequenceClassification',
 'EsmForSequenceClassification',
 'FalconForSequenceClassification',
 'FlaubertForSequenceClassification',
 'FNetForSequenceClassification',
 'FunnelForSequenceClassification',
 'GPT2ForSequenceClassification',
 'GPT2ForSequenceClassification',
 'GPTBigCodeForSequenceClassification',
 'GPTNeoFor

In [21]:
classifier.predict("let us start the class")

[{'label': 'POSITIVE', 'score': 0.999352753162384}]

In [22]:
classifier("the room are smelling badly")

[{'label': 'NEGATIVE', 'score': 0.9996711015701294}]

In [23]:
classifier("I love you")

[{'label': 'POSITIVE', 'score': 0.9998656511306763}]

In [24]:
import pickle

file=open('classifier.pkl','wb')
pickle.dump(classifier,file)

In [26]:
import pickle
import numpy as np
model_file_path = 'classifier.pkl'
with open(model_file_path, 'rb') as file:
    loaded_model = pickle.load(file)

new_data = ('there rooms are not up to standard')


predictions = loaded_model.predict(new_data)

print("Predictions:", predictions)

Predictions: [{'label': 'NEGATIVE', 'score': 0.9994224309921265}]
